In [1]:
!pip install requests beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup

# URL of the sitemap
sitemap_url = "https://wuzzuf.net/sitemap-job-1.xml"

# Fetch the sitemap
response = requests.get(sitemap_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the XML content
    soup = BeautifulSoup(response.content, "xml")
    
    # Find all <loc> tags (which contain the URLs)
    loc_tags = soup.find_all("loc")
    
    # Extract the URLs from the <loc> tags
    urls = [loc.text for loc in loc_tags]
    
    # Print the number of URLs found
    print(f"Found {len(urls)} URLs in the sitemap.")
    
    # Print the first few URLs as a sample
    print("Sample URLs:")
    for url in urls[:5]:  # Print the first 5 URLs
        print(url)
else:
    print(f"Failed to fetch the sitemap. Status code: {response.status_code}")

Found 8706 URLs in the sitemap.
Sample URLs:
https://wuzzuf.net/jobs/p/O2kHWakrqYON-School-Head-of-Admissions-Egypt-Education-Platform-Riyadh-Saudi-Arabia
https://wuzzuf.net/jobs/p/Klp23yDjQD9Q-Senior-Backend-Developer-RocketDevs-Cairo-Egypt
https://wuzzuf.net/jobs/p/gFlUQx2f9TA2-Civil-Engineer-SAAB-Engineering-Consulating-Office-Giza-Egypt
https://wuzzuf.net/jobs/p/3DyZO1fvf2Kp-Senior-Tendering-Engineer-REDCON-Construction-Co-S-A-E-Cairo-Egypt
https://wuzzuf.net/jobs/p/kJMPKoPxaAy0-Senior-Android-Developer-Fixed-Solutions-Cairo-Egypt


In [3]:
# Save the URLs to a file
with open("wuzzuf_job_urls.txt", "w") as file:
    for url in urls:
        file.write(url + "\n")

print("URLs saved to 'wuzzuf_job_urls.txt'.")

URLs saved to 'wuzzuf_job_urls.txt'.


In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import csv

# Function to safely get text from an element (returns empty string if element not found)
def get_element_text(driver, xpath, index=0):
    try:
        elements = driver.find_elements(By.XPATH, xpath)
        if elements and len(elements) > index:
            return elements[index].text
        return ""
    except Exception as e:
        print(f"Error extracting text from {xpath}: {e}")
        return ""

# URL of the sitemap
sitemap_url = "https://wuzzuf.net/sitemap-job-1.xml"

# Fetch the sitemap
response = requests.get(sitemap_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the XML content
    soup = BeautifulSoup(response.content, "xml")
    
    # Find all <loc> tags (which contain the URLs)
    loc_tags = soup.find_all("loc")
    
    # Extract the URLs from the <loc> tags
    urls = [loc.text for loc in loc_tags]
    
    # Print the number of URLs found
    print(f"Found {len(urls)} URLs in the sitemap.")
    
    # Print the first few URLs as a sample
    print("Sample URLs:")
    for url in urls[:5]:  # Print the first 5 URLs
        print(url)
    
    # Initialize the WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    # List to store job data
    data = []
    
    # Navigate to each URL and scrape data
    i=0
    for url in urls[:4]:  # Process the first 2000 URLs for demonstration
        i = i+1
        print(f"Navigating {i}")
        try:
            # Navigate to the URL
            driver.get(url)
            
            # Wait for the page to load (e.g., wait for the job title to appear)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//section/div/h1"))
            )
            
            # Scrape job details
            job_data = {
                "Title": get_element_text(driver, "//section/div/h1"),
                "Company": get_element_text(driver, "//section/div/div/div/a/span"),
                "Working Place": get_element_text(driver, "//section/div/div/a/span"),
                "Location": get_element_text(driver, "//strong/div/a"),
                "Post Date": get_element_text(driver, "//section/div/span"),
                "Number of Positions": get_element_text(driver, "//section/div/div/div/strong"),
                "Applicants": get_element_text(driver, "//div/div/span/span", index=1),
                "Career Level": get_element_text(driver, "//main//section/div/span/span"),
                "Education": get_element_text(driver, "//main//section/div/span/span", index=1),
                "Salary": get_element_text(driver, "//main//section/div/span/span", index=2),
                "Job Category": get_element_text(driver, "//main//section/div/span/span", index=3),
                "Skills": get_element_text(driver, "//div/ul/li/a/span"),
                "Job Description": " ".join([i.text for i in driver.find_elements(By.XPATH, "//div/a/span/span/span")]),

            }
            
            # Append the job data to the list
            data.append(job_data)
        except Exception as e:
            print(f"Error navigating to {url}: {e}")
        finally:
            # Add a delay between requests to avoid overloading the server
            sleep(2)
    
    # Close the WebDriver
    driver.quit()
    
    # Save the scraped data to a CSV file
    with open("wuzzuf_jobs.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=job_data.keys())
        writer.writeheader()  # Write the header row
        writer.writerows(data)  # Write the job data rows
    
    print("Scraped data saved to 'wuzzuf_jobs2.csv'.")
else:
    print(f"Failed to fetch the sitemap. Status code: {response.status_code}")

Found 8571 URLs in the sitemap.
Sample URLs:
https://wuzzuf.net/jobs/p/O2kHWakrqYON-School-Head-of-Admissions-Egypt-Education-Platform-Riyadh-Saudi-Arabia
https://wuzzuf.net/jobs/p/Klp23yDjQD9Q-Senior-Backend-Developer-RocketDevs-Cairo-Egypt
https://wuzzuf.net/jobs/p/gFlUQx2f9TA2-Civil-Engineer-SAAB-Engineering-Consulating-Office-Giza-Egypt
https://wuzzuf.net/jobs/p/3DyZO1fvf2Kp-Senior-Tendering-Engineer-REDCON-Construction-Co-S-A-E-Cairo-Egypt
https://wuzzuf.net/jobs/p/kJMPKoPxaAy0-Senior-Android-Developer-Fixed-Solutions-Cairo-Egypt
Navigating 1
Error navigating to https://wuzzuf.net/jobs/p/O2kHWakrqYON-School-Head-of-Admissions-Egypt-Education-Platform-Riyadh-Saudi-Arabia: HTTPConnectionPool(host='localhost', port=60678): Read timed out. (read timeout=120)
Navigating 2
